<a href="https://colab.research.google.com/github/Marriyam/DataScience/blob/Marriyam/internship2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
upload = files.upload()

Saving loan.csv to loan.csv


In [2]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 55.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a293ceec10df438bcb9625ceec18ed5f4f8f5030bc2d313a44f87480cae10dff
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
#first load all the useful libraries
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
#create the spark session
from pyspark import SparkConf, SparkContext
conf=SparkConf()
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.cores.max", "2")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark = SparkSession.builder.appName("Loan Fraud Detection").config("spark.jars", "flow_utilities/postgresql-42.2.5.jar").getOrCreate()

In [5]:
#Loading data from Colab
df = spark.read.csv('/content/loan.csv',header = True)
df.show()

+--------------------+------------+---------------------+--------------------+---------------------+-------------------------------------------+--------------------+---------------+-------------+---------------+-------------------------------------+--------------------+-------------------+
| loan_application_id|applicant_id|Months_loan_taken_for|             Purpose|Principal_loan_amount|EMI_rate_in_percentage_of_disposable_income|            Property|Has_coapplicant|Has_guarantor|Other_EMI_plans|Number_of_existing_loans_at_this_bank|        Loan_history|high_risk_applicant|
+--------------------+------------+---------------------+--------------------+---------------------+-------------------------------------------+--------------------+---------------+-------------+---------------+-------------------------------------+--------------------+-------------------+
|d68d975e-edad-11e...|     1469590|                    6|electronic equipment|              1169000|                           

**String Indexer for String Values**

In [6]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

S_indexer=StringIndexer(inputCols=['loan_application_id','applicant_id','Months_loan_taken_for','Purpose','Principal_loan_amount','EMI_rate_in_percentage_of_disposable_income','Property','Has_coapplicant','Has_guarantor','Other_EMI_plans','Number_of_existing_loans_at_this_bank','Loan_history'], outputCols=['loan_application_id_indexed','applicant_id_indexed','Months_loan_taken_for_indexed','Purpose_indexed','Principal_loan_amount_indexed','EMI_rate_in_percentage_of_disposable_income_indexed','Property_indexed','Has_coapplicant_indexed','Has_guarantor_indexed','Other_EMI_plans_indexed','Number_of_existing_loans_at_this_bank_indexed','Loan_history_indexed'])
df=S_indexer.fit(df).transform(df)


In [7]:
df.select('Principal_loan_amount', 'Principal_loan_amount_Indexed', 'Has_coapplicant', 'Has_coapplicant_Indexed' ).show()

+---------------------+-----------------------------+---------------+-----------------------+
|Principal_loan_amount|Principal_loan_amount_Indexed|Has_coapplicant|Has_coapplicant_Indexed|
+---------------------+-----------------------------+---------------+-----------------------+
|              1169000|                          8.0|              0|                    0.0|
|              5951000|                        736.0|              0|                    0.0|
|              2096000|                        353.0|              0|                    0.0|
|              7882000|                        849.0|              0|                    0.0|
|              4870000|                        695.0|              0|                    0.0|
|              9055000|                        888.0|              0|                    0.0|
|              2835000|                        491.0|              0|                    0.0|
|              6948000|                        795.0|       

**One-Hot Encoding**

**One Hot Encoding**

In [8]:
OHE=OneHotEncoder(inputCols=['loan_application_id_indexed', 'Purpose_indexed', 'Property_indexed', 'Other_EMI_plans_indexed', 'Loan_history_indexed'], outputCols=['loan_application_id_indexed_OHE', 'Purpose_indexed_OHE', 'Property_indexed_OHE', 'Other_EMI_plans_indexed_OHE', 'Loan_history_indexed_OHE']).fit(df)
O_H_E=OHE.transform(df)

In [9]:
O_H_E.select('Purpose', 'Purpose_indexed', 'Purpose_indexed_OHE').show() 

+--------------------+---------------+-------------------+
|             Purpose|Purpose_indexed|Purpose_indexed_OHE|
+--------------------+---------------+-------------------+
|electronic equipment|            0.0|      (8,[0],[1.0])|
|electronic equipment|            0.0|      (8,[0],[1.0])|
|           education|            5.0|      (8,[5],[1.0])|
|                FF&E|            2.0|      (8,[2],[1.0])|
|         new vehicle|            1.0|      (8,[1],[1.0])|
|           education|            5.0|      (8,[5],[1.0])|
|                FF&E|            2.0|      (8,[2],[1.0])|
|        used vehicle|            3.0|      (8,[3],[1.0])|
|electronic equipment|            0.0|      (8,[0],[1.0])|
|         new vehicle|            1.0|      (8,[1],[1.0])|
|         new vehicle|            1.0|      (8,[1],[1.0])|
|            business|            4.0|      (8,[4],[1.0])|
|electronic equipment|            0.0|      (8,[0],[1.0])|
|         new vehicle|            1.0|      (8,[1],[1.0]

**Vector Assembler**

In [16]:
df

DataFrame[loan_application_id: string, applicant_id: string, Months_loan_taken_for: string, Purpose: string, Principal_loan_amount: string, EMI_rate_in_percentage_of_disposable_income: string, Property: string, Has_coapplicant: string, Has_guarantor: string, Other_EMI_plans: string, Number_of_existing_loans_at_this_bank: string, Loan_history: string, high_risk_applicant: string, loan_application_id_indexed: double, applicant_id_indexed: double, Months_loan_taken_for_indexed: double, Purpose_indexed: double, Principal_loan_amount_indexed: double, EMI_rate_in_percentage_of_disposable_income_indexed: double, Property_indexed: double, Has_coapplicant_indexed: double, Has_guarantor_indexed: double, Other_EMI_plans_indexed: double, Number_of_existing_loans_at_this_bank_indexed: double, Loan_history_indexed: double]

**Vector Assembling to merge features and use them as a single identity**

In [14]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['loan_application_id_indexed','applicant_id_indexed','Months_loan_taken_for_indexed','Purpose_indexed','Principal_loan_amount_indexed','EMI_rate_in_percentage_of_disposable_income_indexed','Property_indexed','Has_coapplicant_indexed','Has_guarantor_indexed','Other_EMI_plans_indexed','Number_of_existing_loans_at_this_bank_indexed','Loan_history_indexed' ] ,outputCol="features")
Vector=assembler.transform(df)


In [15]:
Vector.select('features').show(truncate=False)

Py4JJavaError: ignored

In [ ]:
model_df=Vector.select("features", "high_risk_applicant")

In [ ]:
model_df.show()

**Feature Importance**

To define the important features that are affectig the most

In [13]:
df

DataFrame[loan_application_id: string, applicant_id: string, Months_loan_taken_for: string, Purpose: string, Principal_loan_amount: string, EMI_rate_in_percentage_of_disposable_income: string, Property: string, Has_coapplicant: string, Has_guarantor: string, Other_EMI_plans: string, Number_of_existing_loans_at_this_bank: string, Loan_history: string, high_risk_applicant: string, loan_application_id_indexed: double, applicant_id_indexed: double, Months_loan_taken_for_indexed: double, Purpose_indexed: double, Principal_loan_amount_indexed: double, EMI_rate_in_percentage_of_disposable_income_indexed: double, Property_indexed: double, Has_coapplicant_indexed: double, Has_guarantor_indexed: double, Other_EMI_plans_indexed: double, Number_of_existing_loans_at_this_bank_indexed: double, Loan_history_indexed: double]

In [ ]:
X=df['loan_application_id_indexed','applicant_id_indexed','Months_loan_taken_for_indexed','Purpose_indexed','Principal_loan_amount_indexed','EMI_rate_in_percentage_of_disposable_income_indexed','Property_indexed','Has_coapplicant_indexed','Has_guarantor_indexed','Other_EMI_plans_indexed','Number_of_existing_loans_at_this_bank_indexed','Loan_history_indexed'].toPandas()
# Creating Array structure
y=df.select('high_risk_applicant').toPandas()


In [ ]:
X.shape
y.shape

Separating Trainig and Testing data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.5, random_state =1, stratify = y)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']

categorical

In [ ]:
numerical = [col for col in X_train.columns if X_train[col].dtypes != 'O']

numerical

Filling nulls with mean

In [ ]:
X_test[numerical].isnull().mean()


Nomalizing the data

In [ ]:
y_train.value_counts(normalize=True)*100

In [ ]:
y_test.value_counts(normalize=True)*100

In [ ]:
X_train.isnull().sum()
print()
y_train.isnull().sum()
print()
X_test.isnull().sum()
print()
y_test.isnull().sum()


Using Random Forest Classifier to build model here

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0)


In [ ]:
rfc.fit(X_train, y_train)


In [ ]:
y_pred = rfc.predict(X_test)
rfc_100 = RandomForestClassifier(n_estimators=100, random_state=0)

In [ ]:
rfc_100.fit(X_train, y_train)

In [ ]:
y_pred_100 = rfc_100.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score

print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred_100)))

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)
